# ProPyCore SDK Examples: Documents

This notebook contains snippets from the `documents` submodules: `folders.py` and `files.py`.

In [1]:
import os
import dotenv
import json

import ProPyCore
from importlib import reload
from ProPyCore.access import time
reload(time)

from datetime import datetime

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [2]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [3]:
companies = connection.companies.get()
for company in companies:
    print(company["name"])

company_name = companies[0]["name"]

Rogers-O`Brien Construction


Now get the company details

In [4]:
company = connection.companies.find(identifier=company_name)
projects = connection.projects.get(company_id=company["id"])
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
print(project["name"])

Sandbox Test Project


In [5]:
time_types = connection.time.timecards.get_time_types(
    company_id=company["id"]
)
print(time_types)

[{'id': 980727, 'name': 'Bereavement', 'abbreviated_time_type': 'BER', 'available': True, 'default': False, 'pay_rate': None}, {'id': 980729, 'name': 'FMLA', 'abbreviated_time_type': 'FMLA', 'available': True, 'default': False, 'pay_rate': None}, {'id': 980894, 'name': 'Hourly', 'abbreviated_time_type': 'REG', 'available': True, 'default': False, 'pay_rate': None}, {'id': 89458, 'name': 'Salary', 'abbreviated_time_type': 'SAL', 'available': True, 'default': True, 'pay_rate': None}, {'id': 980728, 'name': 'Sick/Personal', 'abbreviated_time_type': 'SICK/PER', 'available': True, 'default': False, 'pay_rate': None}, {'id': 44298, 'name': 'Vacation', 'abbreviated_time_type': 'VAC', 'available': True, 'default': True, 'pay_rate': None}]


### `time.timecards.get_for_day()`

```python
"""
Returns a list of all daily timecard data for a given project
https://developers.procore.com/reference/rest/timecard-entries?version=latest

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
entry_date : datetime, default None
    date to pull timecards
    None specifies current day
page : int, default 1
    page number
per_page : int, default 100
    number of timecards to include per page

Returns
-------
timecards : list of dict
    available timecard data
"""
```

In [6]:
timecards_by_day = connection.time.timecards.get_for_day(
    company_id=company["id"],
    project_id=project["id"],
    entry_date=datetime(2024, 11, 22)
)

print(f"Number of Timecards: {len(timecards_by_day)}")
# Uncomment to save timecards
#with open("timecards_by_day.json", "w") as f:
#    json.dump(timecards_by_day, f, indent=4))

Number of Timecards: 1


### `time.timecards.get_for_pay_period()`
```python
"""
Return a list of all timecard data for the given pay period
https://developers.procore.com/reference/rest/timesheets?version=latest#list-timecard-data

Parameters
----------
company_id : int
    unique identifier for the company
page : int, default 1
    page number
per_page : int, default 100
    number of timecards to include per page

Returns
-------
timecards : list of dict
    available timecard data
"""
```

In [7]:
timecards_by_pay_period = connection.time.timecards.get_for_day(
    company_id=company["id"],
    project_id=project["id"],
    entry_date=datetime(2024, 11, 22)
)

print(f"Number of Timecards: {len(timecards_by_pay_period)}")
# Uncomment to save timecards
#with open("timecards_by_pay_period.json", "w") as f:
#    json.dump(timecards_by_pay_period, f, indent=4))

ReadTimeout: HTTPSConnectionPool(host='app.procore.com', port=443): Read timed out. (read timeout=None)

### `time.timecards.create_in_project()`
```python
"""
Create a new timecard
https://developers.procore.com/reference/rest/timecard-entries?version=latest#create-timecard-entry-project

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    Procore's unique identifier for the project
data : dict
    timecard data to create
"""
```

In [8]:
sandbox_project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
data = {
    "hours": 8,
    "date": "2025-01-27",
    "datetime": "2025-01-27T12:00:00Z",
}

connection.time.timecards.create_in_project(
    company_id=company["id"],
    project_id=sandbox_project["id"],
    data=data
)

Response Status Code: 400
Response Text: {"errors":"param is missing or the value is empty: timecard_entry"}


ProcoreException: 'Error: 400'

---